## 1.FM背景
&emsp;&emsp;判断一个物品是否进行推荐，需要根据ctr的预估的点击率排序决定，常用的方法有人工特征工程+LR，GBDT + LR，FM，FFM模型。

## 2.FM原理
&emsp;&emsp;FM是因子分解机，主要解决稀疏数据下的特征组合问题，并且其预测的复杂度是线性的，对于连续和离散特征有较好的通用性。在实际中，很多categorical特征，经过one-hot编码后，导致样本的数据变得很稀疏。

### 2.1.特征交叉
&emsp;&emsp;普通的线性模型，都是将各个特征独立考虑，并没有考虑到特征与特征之间的相互关系，但是实际上，特征之间可能具有一定的相关性。例如新闻推荐，男性看的军事类别新闻较多，女性看的情感类别新闻较多，可以看出性别和新闻的类别具有一定的关联性，如果能找出这类的特征，可能对模型的提升非常明显，简单起见，目前只考虑二阶交叉的情况。模型如下：
$$y(x) = w_{0} + \sum_{i=1}^{n}{w_{i}x_{i}} + \sum_{i=1}^{n}\sum_{j=i+1}^{n}{w_{ij}x_{i}x_{j}} \tag{1}$$

&emsp;&emsp;其中，n代表样本的特征数量，$x_{i}$是第i个特征的值，$w_{0},w_{i},w_{ij}$是模型参数，只有当$x_{i}$,$x_{j}$都不为0的时候，交叉才有意义，在数据稀疏的情况下，满足交叉项不为0的样本将非常少，当训练样本不足时，很容易导致参数$w_{ij}$训练不从分而不准确，最终影响模型效果。那么交叉项参数的训练问题可以用矩阵分解来近视解决，有下面的公式。
$$y(x) = w_{0} + \sum_{i=1}^{n}{w_{i}x_{i}} + \sum_{i=1}^{n}\sum_{j=i+1}^{n}{<v_{i},v_{j}>x_{i}x_{j}} \tag{2}$$

&emsp;&emsp;其中模型需要估计的参数是:
$$w_{0}\in R,w \in R^{n},V \in R^{n*k}$$

&emsp;&emsp;$<.,.>$是两个k维向量的内积：
$$<v_{i},v_{j}> = \sum_{f=1}^{k}{v_{i,f}v_{j,f}}\tag{3}$$

&emsp;&emsp;对任意正定矩阵W，只要k足够大，就存在矩阵W,使得$W = VV^{T}$，然而在数据稀疏的情况下，应该选择较小的k，因为没有足够的数据来估计$w_{ij}$，限制k的大小提高了模型更好的泛化能力。

&emsp;&emsp;为什么说可以提高模型的泛化能力？

&emsp;&emsp;首先在训练数据不变的情况下，需要预测的参数变少，可以一定程度提高准确性，之前需要的参数是(n * n /2)，改为因子分解后需要预测的参数是(n * f)，其中f远小于n，因此可以提升参数的准确性。

&emsp;&emsp;直接计算公式(2)的复杂度是$O(kn^{2})$，因为所有的交叉特征都需要计算，但是通过公式变换，可以减少到线性复杂度，推理如下:

![推理](./pic/inference.svg)

&emsp;&emsp;可以看出此时的时间复杂度是$O(kn)$

### 2.2 预测
&emsp;&emsp;FM算法可以应用在多种的预测任务中，包括:

1. regression：用最小平方误差进行优化
2. binary classfication：使用hinge loss 或者logit loss进行优化

&emsp;&emsp;对以上的任务中，正则化项参数一般加入目标函数以防止过拟合。

### 2.3 参数学习
&emsp;&emsp;从上面的描述中可以知道FM可以在线性的时间内进行预测，因此模型的参数可以通过梯度下降的方法来学习，对于各种损失函数，FM的模型梯度是:

![update](./pic/update.svg)

&emsp;&emsp;由于$\sum_{j=1}^{n}{v_{j,f}x_{j}}$只与f有关，与i是是独立的，可以提前计算出来，并且每次梯度更新可以在常数时间复杂度内完成，因此FM参数训练的复杂度也是$O{kn}$。综上可知，FM可以在线性时间训练和预测，是一种非常高效的模型。

### 2.4 总结

&emsp;&emsp;FM模型有两个优势:
1. 在高度稀疏的情况下特征之间的交叉仍然能够估计，而且可以泛化到未被观察的交叉。
2. 参数的学习和模型的预测的时间复杂度是线性的。

&emsp;&emsp;FM的优化地方
1. 特征为全交叉，耗费资源，通常user与user,item与item内部的交叉作用要小于user与item的交叉。
2. 使用矩阵计算，而不是for循环计算。
3. 可以探索高阶交叉特征的构造。

## 3.FM实践
&emsp;&emsp;数据分析，数据包括u.item,u.user,ua.base,ua.test
1. u.item包括的数据格式如下:
    movie id | movie title | release date | video release date |
    IMDb URL | unknown | Action | Adventure | Animation |
    Children's | Comedy | Crime | Documentary | Drama | Fantasy |
    Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
    Thriller | War | Western |

2. u.user包括的数据如下:
    user id | age | gender | occupation | zip code

3. ua.base和ua.test的数据格式如下:
    user id | item id | rating | timestamp
    
### 3.1 分析u.item数据

In [2]:
# 分析数据
!head ../data/u.item

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0
6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0
8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Babe%20(1995)|0|0|0|0|1

In [3]:
item_path = '../data/u.item'
import pandas as pd
item_columns = ['item_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', 'Children',
            'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
            'Thriller', 'War', 'Western']
df_item = pd.read_csv(item_path,sep = '|',names=item_columns,encoding='ISO-8859-1')
# 抛弃一些没用的列
df_item.drop(columns=['title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown'],inplace=True)

In [4]:
df_item.head()

,item_id,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


### 3.2 分析u.user数据

In [5]:
!head ../data/u.user

1|24|M|technician|85711
2|53|F|other|94043
3|23|M|writer|32067
4|24|M|technician|43537
5|33|F|other|15213
6|42|M|executive|98101
7|57|M|administrator|91344
8|36|M|administrator|05201
9|29|M|student|01002
10|53|M|lawyer|90703


In [11]:
user_path = '../data/u.user'
user_columns = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
df_user = pd.read_csv(user_path,sep = '|',names=user_columns)
df_user.head()

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [12]:
# 给age分类
df_user['age'] = pd.cut(df_user['age'], [0,10,20,30,40,50,60,70,80,90,100], labels=['0-10','10-20','20-30','30-40','40-50','50-60','60-70','70-80','80-90','90-100'])

In [13]:
df_user.head()

,user_id,age,gender,occupation,zip_code
0,1,20-30,M,technician,85711
1,2,50-60,F,other,94043
2,3,20-30,M,writer,32067
3,4,20-30,M,technician,43537
4,5,30-40,F,other,15213


In [14]:
df_user = pd.get_dummies(df_user, columns=['gender', 'occupation', 'age'])
df_user.drop(columns=['zip_code'],inplace=True)

In [15]:
df_user.head()

,user_id,gender_F,gender_M,occupation_administrator,occupation_artist,occupation_doctor,occupation_educator,occupation_engineer,occupation_entertainment,occupation_executive,...,age_0-10,age_10-20,age_20-30,age_30-40,age_40-50,age_50-60,age_60-70,age_70-80,age_80-90,age_90-100
0,1,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,3,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,4,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,5,1,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
